# [Inconsistent Data Entry](https://www.kaggle.com/code/alexisbcook/inconsistent-data-entry)

In [1]:
import pandas as pd

import numpy as np
np.random.seed(0)

# Helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

In [2]:
professors = pd.read_csv('pakistan_intellectual_capital.csv')

## Preliminary text pre-processing

In [3]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


### Cleaning up the `Country` column

In [4]:
# Get all unique values from the 'Country' column
countries = professors['Country'].unique()

# Sort countries alphabetically
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

Looking at this, we can see some problems due to inconsistent data entry: `' Germany'` and `'germany'` or `' New Zealand'` and `'New Zealand'`. <br>
The first thing we will do is make everything lower case (can changed it back at the end\) and remove any white spaces at the beginning and end of cells. <br>
Inconsistencies in capitalizations and white spaces are very common in text data and you can fix a good percentage of your text data entry inconsistencies by doing this.

In [5]:
# Convert to lower case
professors['Country'] = professors['Country'].str.lower()

# Remove leading and trailing white spaces
professors['Country'] = professors['Country'].str.strip()

## Fuzzy matching to correct inconsistent data entry

In [6]:
# Get all unique values from the 'Country' column
countries = professors['Country'].unique()

# Sort countries alphabetically
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

It does look like there is another inconsistency: `'southkorea'` and `'south korea'` should be the same. <br>

We're going to use the `fuzzywuzzy` package to help identify which strings are closest to each other. <br>

>Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. <br>
In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "south korea".

In [7]:
# Get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('austria', 33),
 ('ireland', 33),
 ('pakistan', 32),
 ('portugal', 32),
 ('scotland', 32),
 ('australia', 30)]

We can see that two of the items in the cities are very close to "south korea": "south korea" and "southkorea". <br>
Let's replace all rows in our `"Country"` column that have a ratio of > `47` with `"south korea"`. <br>

To do this, we will write a function.

In [8]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    '''
    Function to replace rows in the provided column of the provided dataframe
    that match the provided string above the provided ratio with the provided string
    '''
    
    # Get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to the input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > min_ratio
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function is done
    print("Done!")

In [9]:
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

Done!


Now let's check the unique values in the "Country" column again and make sure we've tidied up "south korea" correctly.

In [10]:
countries = professors['Country'].unique()
countries.sort()

countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)